In [48]:
!pip install ipywidgets
!pip install datasets
!jupyter nbextension enable --py widgetsnbextension
!pip install --upgrade transformers
!pip install transformers datasets accelerate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)
import torch


In [51]:
# Load the synthetic dataset
with open("../data/synthetic_dataset_v1.json") as f:
    data = json.load(f)

# Convert to HF dataset format
samples = [{"text": f"Suggest domain names for: {item['business_description']}"} for item in data]
hf_dataset = Dataset.from_list(samples)
hf_dataset = hf_dataset.train_test_split(test_size=0.1)

print(f"Train size: {len(hf_dataset['train'])}")
print(f"Test size: {len(hf_dataset['test'])}")
print(hf_dataset['train'][0])


Train size: 180
Test size: 20
{'text': 'Suggest domain names for: family law firm with a global audience'}


In [52]:
model_id = "sshleifer/tiny-gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id)
model.config.pad_token_id = tokenizer.pad_token_id


In [53]:
def tokenize(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=64,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = hf_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [54]:
import transformers
print(transformers.__version__)


4.53.0


In [55]:
training_args = TrainingArguments(
    output_dir="../models/checkpoints/tiny-gpt2",
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_steps=10,
    do_eval=True,
    save_steps=500,
    eval_steps=500,
    seed=42,
    fp16=torch.cuda.is_available()
)



In [56]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


In [57]:
trainer.train()


/Users/liliakryvelova/Course_Python/domain_suggestion/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,10.643800
20,10.639500
30,10.636500
40,10.634600
50,10.633100
60,10.631600
70,10.630200
80,10.628900
90,10.628100
100,10.627100


TrainOutput(global_step=115, training_loss=10.632441678254501, metrics={'train_runtime': 8.8748, 'train_samples_per_second': 101.411, 'train_steps_per_second': 12.958, 'total_flos': 52531200.0, 'train_loss': 10.632441678254501, 'epoch': 5.0})

In [58]:
model.save_pretrained("../models/checkpoints/tiny-gpt2")
tokenizer.save_pretrained("../models/checkpoints/tiny-gpt2")


('../models/checkpoints/tiny-gpt2/tokenizer_config.json',
 '../models/checkpoints/tiny-gpt2/special_tokens_map.json',
 '../models/checkpoints/tiny-gpt2/vocab.json',
 '../models/checkpoints/tiny-gpt2/merges.txt',
 '../models/checkpoints/tiny-gpt2/added_tokens.json',
 '../models/checkpoints/tiny-gpt2/tokenizer.json')

In [59]:
def generate_domain(prompt: str, max_new_tokens=20):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Try it
prompt = "Suggest domain names for: online education platform for kids"
print(generate_domain(prompt))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Suggest domain names for: online education platform for kids Dreamsshows Lateshows Late Singapore boils lined perhaps TelevisionSexual workshopsSexual448 membershipacious workshops653 Boone mutual
